# Fraud Detection Analytics

This notebook demonstrates advanced fraud detection using graph analytics on Amazon Neptune. Using the synthetic fraud data loaded in the previous notebook, we'll explore sophisticated fraud patterns, analyze shell account networks, detect money laundering rings, and identify synthetic identity fraud using powerful Gremlin graph queries.

## Fraud Detection Workflow

```mermaid
graph TD
    A[Neptune Graph Database] --> B[Data Verification]
    B --> C[Fraud Pattern Detection]
    C --> D[Shell Account Analysis]
    D --> E[Money Laundering Detection]
    E --> F[Synthetic Identity Fraud]
    F --> G[Advanced Analytics]
    G --> H[Risk Assessment]
    
    C --> C1[Fraud Types: 9 patterns]
    D --> D1[Shell Companies: 100]
    E --> E1[Circular Transactions]
    F --> F1[Synthetic Accounts: 50]
    G --> G1[Network Analysis]
    H --> H1[Risk Scores ≥80]
    
    style A fill:#f3e5f5
    style E fill:#ffebee
    style F fill:#fff3e0
    style H fill:#e8f5e8
```

## Analytics Capabilities
- **Pattern Recognition**: Identify sophisticated fraud schemes
- **Network Analysis**: Map relationships between fraudulent entities
- **Risk Scoring**: Quantify transaction and account risk levels
- **Timeline Analysis**: Track fraud evolution over time
- **Institution Impact**: Assess fraud exposure across financial institutions

**Prerequisites:** Run `Enhanced_Fraud_Bulk_Load_Workflow.ipynb` first to load data into Neptune.

## Setup

In [ ]:
%load_ext graph_notebook.magics
%graph_notebook_version

### Retrieve Neptune Endpoint from CloudFormation

Get the Neptune endpoint from the `neptune-cluster` CloudFormation stack output.
Should be of format: `financial-network-cluster.cluster-xxx.us-west-2.neptune.amazonaws.com`

In [ ]:
%%graph_notebook_config
{
  "host": "financial-network-cluster.cluster-xxx.us-west-2.neptune.amazonaws.com",
  "neptune_service": "neptune-db",
  "port": 8182,
  "auth_mode": "DEFAULT",
  "ssl": true,
  "ssl_verify": true,
  "aws_region": "us-west-2",
  "load_from_s3_arn": ""
}

In [ ]:
print("🔍 Fraud Detection Analytics - Ready!")
print("\n📊 This notebook includes:")
print("   • Data verification queries")
print("   • Fraud pattern detection")
print("   • Shell account analysis")
print("   • Money laundering detection")
print("   • Synthetic identity fraud")
print("   • Advanced fraud analytics")

## Step 1: Verify Data in Neptune

### Check total vertex count

In [ ]:
%%gremlin
g.V().count()

### Check total edge count

In [ ]:
%%gremlin
g.E().count()

### Check vertex labels

In [ ]:
%%gremlin
g.V().label().groupCount()

### Check edge labels

In [ ]:
%%gremlin
g.E().label().groupCount()

## Step 2: Fraud Pattern Detection

### Find fraud patterns

In [ ]:
%%gremlin
g.V().hasLabel('FraudPattern').valueMap()

### Count fraud transactions by type

In [ ]:
%%gremlin
g.E().hasLabel('PAYMENT')
 .has('is_fraud', true)
 .groupCount()
 .by('fraud_type')

## Step 3: Shell Account Analysis

### Find shell accounts and their transactions

In [ ]:
%%gremlin
g.V().has('Account', 'is_shell', true)
 .as('shell')
 .bothE('PAYMENT')
 .as('transaction')
 .select('shell', 'transaction')
 .by(id())
 .by(valueMap())
 .limit(10)

### Shell account transaction volume

In [ ]:
%%gremlin
g.V().has('Account', 'is_shell', true)
 .project('shell_account', 'total_transactions', 'total_amount')
 .by(id())
 .by(bothE('PAYMENT').count())
 .by(bothE('PAYMENT').values('amount').sum())
 .order().by(select('total_amount'), desc)
 .limit(10)

### Shell account network visualization

In [ ]:
%%gremlin
g.V().has('Account', 'is_shell', true)
 .outE('PAYMENT').has('is_fraud', true)
 .inV()
 .path()
 .by(id())
 .by(valueMap('amount', 'fraud_type'))
 .limit(15)

## Step 4: High-Risk Fraud Transactions

### Find high-risk fraud transactions

In [ ]:
%%gremlin
g.E().hasLabel('PAYMENT')
 .has('is_fraud', true)
 .has('risk_score', gte(80))
 .project('from', 'to', 'amount', 'fraud_type', 'risk_score')
 .by(outV().id())
 .by(inV().id())
 .by('amount')
 .by('fraud_type')
 .by('risk_score')
 .limit(20)

### Risk score distribution

In [ ]:
%%gremlin
g.E().hasLabel('PAYMENT')
 .has('is_fraud', true)
 .groupCount()
 .by('risk_score')
 .order(local).by(keys, asc)

### High-risk fraud network visualization

In [ ]:
%%gremlin
g.E().hasLabel('PAYMENT')
 .has('is_fraud', true)
 .has('risk_score', gte(90))
 .limit(20)
 .bothV()
 .path()
 .by(id())
 .by(valueMap('amount', 'risk_score', 'fraud_type'))

## Step 5: Money Laundering Detection

### Find money laundering patterns (circular transactions)

In [ ]:
%%gremlin
g.V().as('start')
 .outE('PAYMENT').has('fraud_type', 'money_laundering_ring')
 .inV().as('middle')
 .outE('PAYMENT').has('fraud_type', 'money_laundering_ring')
 .inV().as('end')
 .where('start', eq('end'))
 .path()
 .by(id())
 .by('amount')
 .limit(10)

### Money laundering ring analysis

In [ ]:
%%gremlin
g.E().hasLabel('PAYMENT')
 .has('fraud_type', 'money_laundering_ring')
 .project('transaction_count', 'total_amount', 'avg_amount')
 .by(count())
 .by(values('amount').sum())
 .by(values('amount').mean())

## Step 6: Synthetic Identity Fraud

### Find synthetic identity fraud patterns

In [ ]:
%%gremlin
g.V().has('Account', 'is_synthetic', true)
 .as('synthetic')
 .outE('PAYMENT').has('is_fraud', true)
 .as('fraud_tx')
 .inV().as('receiver')
 .select('synthetic', 'fraud_tx', 'receiver')
 .by(id())
 .by(valueMap('amount', 'fraud_type', 'risk_score'))
 .by(id())
 .limit(10)

### Synthetic account fraud statistics

In [ ]:
%%gremlin
g.V().has('Account', 'is_synthetic', true)
 .project('synthetic_account', 'fraud_transactions', 'fraud_amount')
 .by(id())
 .by(outE('PAYMENT').has('is_fraud', true).count())
 .by(outE('PAYMENT').has('is_fraud', true).values('amount').sum())
 .order().by(select('fraud_amount'), desc)
 .limit(10)

### Synthetic identity network visualization

In [ ]:
%%gremlin
g.V().has('Account', 'is_synthetic', true)
 .bothE('PAYMENT').has('is_fraud', true)
 .limit(25)
 .bothV()
 .path()
 .by(id())
 .by(valueMap('amount', 'fraud_type'))

## Step 7: Advanced Network Analytics

### Institution network analysis

In [ ]:
%%gremlin
g.V().hasLabel('Institution')
 .project('institution', 'type', 'account_count', 'transaction_count')
 .by('name')
 .by('type')
 .by(in('BELONGS_TO').count())
 .by(in('BELONGS_TO').bothE('PAYMENT').count())
 .order().by(select('transaction_count'), desc)

### Institution network visualization

In [ ]:
%%gremlin
g.V().hasLabel('Institution').limit(5)
 .in('BELONGS_TO').limit(20)
 .bothE('PAYMENT').limit(50)
 .bothV()
 .path()

### High-transaction institution networks

In [ ]:
%%gremlin
g.V().hasLabel('Institution')
 .where(in('BELONGS_TO').bothE('PAYMENT').count().is(gte(1000)))
 .limit(2)
 .in('BELONGS_TO').limit(15)
 .bothE('PAYMENT').limit(30)
 .bothV()
 .path()

### Find accounts with highest fraud transaction counts

In [ ]:
%%gremlin
g.V().hasLabel('Account')
 .where(bothE('PAYMENT').has('is_fraud', true).count().is(gte(3)))
 .project('account', 'fraud_count', 'total_fraud_amount')
 .by(id())
 .by(bothE('PAYMENT').has('is_fraud', true).count())
 .by(bothE('PAYMENT').has('is_fraud', true).values('amount').sum())
 .order().by(select('fraud_count'), desc)
 .limit(10)

### Find institutions with highest fraud exposure

In [ ]:
%%gremlin
g.V().hasLabel('Institution')
 .as('institution')
 .in('BELONGS_TO')
 .bothE('PAYMENT').has('is_fraud', true)
 .groupCount()
 .by(select('institution').by(id()))
 .order(local).by(values, desc)
 .limit(local, 10)

### Fraud network connectivity analysis

In [ ]:
%%gremlin
g.V().has('Account', 'is_shell', true)
 .as('shell')
 .both('PAYMENT')
 .where(neq('shell'))
 .groupCount()
 .by(id())
 .order(local).by(values, desc)
 .limit(local, 10)

## Step 8: Fraud Timeline Analysis

### Fraud transactions by time period

In [ ]:
%%gremlin
g.E().hasLabel('PAYMENT')
 .has('is_fraud', true)
 .groupCount()
 .by('timestamp')
 .order(local).by(keys, asc)
 .limit(local, 20)

### Average fraud amount by fraud type

In [ ]:
%%gremlin
g.E().hasLabel('PAYMENT')
 .has('is_fraud', true)
 .group()
 .by('fraud_type')
 .by(values('amount').mean())
 .order(local).by(values, desc)

## Summary

✅ **Analytics Completed:**
1. Verified data integrity in Neptune
2. Analyzed fraud patterns and distributions
3. Identified shell account networks
4. Detected money laundering rings
5. Found synthetic identity fraud
6. Performed advanced fraud analytics

🎯 **Key Insights:**
- High-risk transactions with risk scores ≥80
- Circular money laundering patterns
- Shell account transaction volumes
- Institution fraud exposure levels
- Fraud network connectivity patterns

🔍 **Next Steps:**
- Build ML models for real-time fraud detection
- Create fraud risk scoring algorithms
- Develop automated alert systems
- Implement graph-based anomaly detection